In [ ]:
# CodeArtifactへのログインと、CodeArtifactを経由したpip installです
# <ACCOUNTID>の部分を12桁のAWSアカウントに差し替えてから実行してください
!aws codeartifact login --tool pip --domain sagemakergovernancestack-domain --domain-owner <ACCOUNTID> --repository sagemakergovernancestack-domain
%pip install uuid

In [ ]:
import sagemaker
import boto3
from sklearn import datasets
import pandas as pd

# CDKによってデプロイされた次のリソースを設定してから実行してください。
# BUCKET_NAME: ユーザ専用のバケット名
# USER_NAME: ユーザプロファイル名

BUCKET_NAME= '<YOUR BUCKET>'
USER_NAME = '<YOUR USER NAME>'


iris = datasets.load_iris()
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
df['target'] = iris.target

df.to_csv("iris.csv", index=False)
sagemaker_session = sagemaker.Session(default_bucket=BUCKET_NAME)
input_data = sagemaker_session.upload_data(path="iris.csv", key_prefix='data/iris')

In [ ]:
from sagemaker.pytorch import PyTorch
from sagemaker.image_uris import retrieve
import uuid


region = sagemaker_session.boto_session.region_name

uri = retrieve(framework='pytorch',region=region,version='1.12.0',py_version='py38',image_scope='training', instance_type='ml.m5.large')

output_path = "s3://" + BUCKET_NAME + "/output"
estimator = PyTorch(entry_point="train.py",
                    role=sagemaker.get_execution_role(),
                    image_uri=uri,
                    instance_count=1,
                    instance_type="ml.m5.large",
                    hyperparameters={},
                    sagemaeker_session=sagemaker_session,
                    output_path=output_path,
                    enable_network_isolation=True)
job_name = f"{USER_NAME}-{uuid.uuid4()}"

estimator.fit({'train': input_data}, job_name=job_name)